# Run the tracker

This jupyter notebook bundles all necessary high-level steps to train the tracker. We start with running the simulation to generate training data.

In [1]:
from tqdm import tqdm
import pickle
import sys
import numpy as np
from datetime import datetime
import os
sys.path.append('build/')
import simulation
from time import time, sleep
from python.plotter import Plotter
from python.GNNtracker import GNNTracker
from python.constants import *

Now we need to get simulation data of the warehouse. We can use the simulation to generate new simulation data. This cell is optional, we can also load old simulation data.

In [2]:
T_simulation = 120 # 1 hour

sim = simulation.Simulation(
    T_step=0.1, 
    N_humans=3, 
    N_robots=1
)
sim_states = []
N_minutes = int(T_simulation / 60)
N_hours = int(T_simulation / 3600)
pbar = tqdm(range(0, N_minutes), desc='Simulation')
for i in pbar:
    sim_states += sim.step(int(60 / sim.T_step))
    current_hour = int(i / 60)
    current_minute = i % 60
    pbar.set_postfix({'Simulated time': f'{current_hour}:{current_minute} of {N_hours} hours'})

filename = os.path.join(LOG_FOLDER, 'log_' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + '.pkl')
with open(filename, 'wb') as outp:
    pickle.dump({
        'T_step': sim.T_step,
        'T_simulation': np.arange(0, len(sim_states) * sim.T_step, sim.T_step),
        'N_humans': sim.N_humans,
        'N_robots': sim.N_robots,
        'sim_states': sim_states,
    }, outp, pickle.HIGHEST_PROTOCOL)
    

Simulation: 100%|██████████| 2/2 [00:00<00:00, 52.56it/s, Simulated time=0:1 of 0 hours]


Now we load the generated simulation logs.

In [3]:
if "filename" not in locals():
    filename = os.path.join(LOG_FOLDER, 'log_2024-08-16_20-34-51.pkl')
with open(filename, 'rb') as f:
    sim_log = pickle.load(f)
sim_states = sim_log['sim_states']
T_simulation = sim_log['T_simulation']
T_step = sim_log['T_step']
N_humans = sim_log['N_humans']
N_robots = sim_log['N_robots']
# load simulation to have access to member utility functions
sim = simulation.Simulation(
    T_step=0.1, 
    N_humans=3, 
    N_robots=1
)

Now that we have the simulation data, we can run the actual tracker.

In [4]:
plot = True
record_video = False
realtime = True

tracker = GNNTracker(
    T_step=T_step,
    train=False,
    # use extend_perception implemented in simulation class with C++ for speedup
    extend_perception=sim.extend_perception 
)

if plot: plotter = Plotter(record_frames=record_video)

for sim_state in sim_states:
    start = time()

    # outer list: robots, inner list: perceived humans for every robot
    robot_perceptions = [{ 
        "ego_position": agent['ego_position'],
        "perceived_humans": agent['perceived_humans'] 
    } for agent in sim_state if agent['type'] == 'robot']

    tracker.add_observation(robot_perceptions)
    node_probabilities = tracker.predict()

    if plot: plotter.update(sim_state, node_probabilities)    
    
    if realtime: sleep(max(0, T_step - (time() - start)))

if record_video: plotter.create_video(T_step)

invalid command name "140693066012480process_stream_events"
    while executing
"140693066012480process_stream_events"
    ("after" script)
can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


KeyboardInterrupt: 